# Chapter 7: Distributed Processing with Ray

**Data-Juicer User Guide**

- Git Commit: `v1.4.5`
- Commit Date: 2026-01-16
- Repository: https://github.com/datajuicer/data-juicer

# Table of Contents

1. [Setup](#setup)
2. [Explore Demo Configurations](#explore-demo-configurations)
3. [Run Distributed Processing with Demo Config](#run-distributed-processing-with-demo-config)
4. [Monitor Resources](#monitor-resources)
5. [Ray Dashboard](#ray-dashboard)
6. [Multi-Node Cluster Setup](#multi-node-cluster-setup)
7. [Check Processing Results](#check-processing-results)
8. [Try Deduplication Demo](#try-deduplication-demo)
9. [Performance Tips](#performance-tips)
10. [Cleanup](#cleanup)
11. [Further Reading](#further-reading)

## Setup 

### Clone Data-Juicer Repository

First, let's clone the Data-Juicer repository to access the demo configurations and data:

In [ ]:
!git clone --depth 1 https://github.com/datajuicer/data-juicer.git

In [ ]:
# Install Data-Juicer with Ray support
!uv pip install py-data-juicer[distributed]

### Setup Ray Cluster

In [ ]:
# To start a local Ray cluster, run this command in your terminal:
# !ray start --head

In [ ]:
# Check Ray cluster status
!ray status

## Explore Demo Configurations

Data-Juicer provides ready-to-use demo configurations in `demos/process_on_ray/`:

In [ ]:
# List available demo configs
!ls -lh data-juicer/demos/process_on_ray/configs/

In [ ]:
# View the demo configuration
!cat data-juicer/demos/process_on_ray/configs/demo.yaml

## Run Distributed Processing with Demo Config

Now let's run the distributed processing using the demo configuration:

In [ ]:
# Process with Ray using demo config
!cd data-juicer && dj-process --config demos/process_on_ray/configs/demo.yaml

## Monitor Resources

In [ ]:
# Check resource usage
import ray
from data_juicer.utils.ray_utils import ray_cpu_count, ray_gpu_count

ray.init()

print(f"Total CPUs: {ray_cpu_count()}")
print(f"Total GPUs: {ray_gpu_count()}")

## Ray Dashboard

Access Ray Dashboard at: `http://localhost:8265`

The dashboard provides:
- Real-time resource utilization
- Task execution timeline
- Memory usage statistics
- Error logs and debugging info

## Multi-Node Cluster Setup

In [ ]:
print("Multi-node Ray cluster setup:")
print("""
# On head node:
ray start --head --port=6379 --num-cpus=8

# On worker nodes:
ray start --address='<head-node-ip>:6379' --num-cpus=8

# In Data-Juicer config:
executor_type: 'ray'
ray_address: '<head-node-ip>:6379'
""")

## Check Processing Results

In [ ]:
# Check output directory
!ls -lh data-juicer/outputs/demo/

In [ ]:
# View sample processed data
import os
import json

output_dir = 'data-juicer/outputs/demo/demo-processed'
try:
    sample_files = os.listdir(output_dir)
    print(f"Sample files count: {len(sample_files)}")
    for sample_file in sample_files:
        with open(os.path.join(output_dir, sample_file), 'r') as f:
            print(f"Sample file: {sample_file}")
            print(json.dumps(json.load(f), indent=4))
except FileNotFoundError:
    print("Output directory not found")
    

## Try Deduplication Demo

Data-Juicer also provides a deduplication demo using Ray:

In [ ]:
# View deduplication config
!cat data-juicer/demos/process_on_ray/configs/dedup.yaml

In [ ]:
# check input directory
!ls -lh data-juicer/demos/process_on_ray/data

In [ ]:
# Run deduplication
!cd data-juicer && dj-process --config demos/process_on_ray/configs/dedup.yaml

In [ ]:
# Check output directory
!ls -lh data-juicer/outputs/demo-dedup/demo-ray-bts-dedup-processed

In [ ]:
# View sample processed data
import os
import json
output_dir = 'data-juicer/outputs/demo-dedup/demo-ray-bts-dedup-processed'
try:
    sample_files = os.listdir(output_dir)
    print(f"Sample files count: {len(sample_files)}")
    for sample_file in sample_files:
        with open(os.path.join(output_dir, sample_file), 'r') as f:
            for i, line in enumerate(f):
                if i < 3:
                    print(json.dumps(json.loads(line), ensure_ascii=False))
except FileNotFoundError:
    print("Output directory not found")

## Performance Tips

Performance optimization tips for Ray processing:

1. **Shard Size**: Adjust export_shard_size based on dataset size
   - Smaller shards (100-1000): Better for fault tolerance
   - Larger shards (5000-10000): Better for throughput

2. **Caching**: Enable caching for repeated operations
   use_cache: true
   cache_compress: 'gzip'

3. **Operator Fusion**: Combine compatible operators
   op_fusion: true

4. **Resource Allocation**: Match workers to available resources
   - CPU-bound ops: More workers
   - GPU-bound ops: Fewer workers with GPU allocation

5. **Monitoring**: Use Ray Dashboard at http://localhost:8265

## Cleanup

In [ ]:
# Stop Ray cluster
# !ray stop

In [ ]:
# Remove cloned Data-Juicer repository
!rm -rf data-juicer

## Further Reading

- [Distributed Processing Documentation](https://datajuicer.github.io/data-juicer/en/main/docs/Distributed.html)
- [Ray Documentation](https://docs.ray.io/)